# 01 - Análisis Exploratorio de Datos

**Proyecto:** Forecast Promtur - Tráfico Orgánico  
**Objetivo:** Explorar y entender la estructura de los datos de GA4

---

## Contenido:
1. Carga de datos
2. Configuración de nombres de columnas
3. Inspección inicial
4. Calidad de datos
5. Análisis por canal
6. Visualizaciones exploratorias
7. Conclusiones preliminares

## 1. Configuración inicial y librerías

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Semilla para reproducibilidad
np.random.seed(42)

print("✅ Librerías importadas correctamente")

## 2. Carga de datos

In [ ]:
# Definir rutas del proyecto
DATA_RAW = Path('../data/raw')
DATA_PROCESSED = Path('../data/processed')
RESULTS_FIGURES = Path('../results/figures/exploratory')

# Crear carpetas si no existen
RESULTS_FIGURES.mkdir(parents=True, exist_ok=True)

# Cargar dataset
csv_file = DATA_RAW / 'ga4_promtur_organic_2025.csv'

# Verificar que existe el archivo
if csv_file.exists():
    df_raw = pd.read_csv(csv_file)
    print(f"✅ Dataset cargado exitosamente")
    print(f"📊 Dimensiones: {df_raw.shape[0]} filas x {df_raw.shape[1]} columnas")
else:
    print(f"❌ Error: No se encontró el archivo {csv_file}")
    print(f"📁 Asegúrate de colocar el CSV en: {DATA_RAW}")

## 2.1 Configuración de nombres de columnas

**IMPORTANTE:** Aquí se definen los nombres EXACTOS de las columnas del CSV original.  
Si cambias el CSV en el futuro, solo modifica esta celda.

In [ ]:
# ===================================================================
# CONFIGURACIÓN: Nombres de columnas del CSV original
# ===================================================================
# Estos nombres deben coincidir EXACTAMENTE con los del CSV

year_col = 'Year'
month_col = 'Month number'  # Nota: 'number' con n minúscula
canal_col = 'Session Default Channel Group Custom (Recovery)'
sessions_col = 'Sessions - GA4'
bounces_col = 'Bounces'
duration_col = 'Total session duration - GA4'
views_col = 'Views - GA4'

# Verificar que todas las columnas existen en el dataset
columnas_requeridas = [
    year_col, month_col, canal_col, sessions_col, 
    bounces_col, duration_col, views_col
]

columnas_faltantes = [col for col in columnas_requeridas if col not in df_raw.columns]

if columnas_faltantes:
    print("❌ ERROR: Las siguientes columnas no se encontraron en el CSV:")
    for col in columnas_faltantes:
        print(f"   - {col}")
    print("\n📋 Columnas disponibles en el CSV:")
    for col in df_raw.columns:
        print(f"   - {col}")
else:
    print("✅ Variables de columnas configuradas correctamente")
    print(f"   - Año: '{year_col}'")
    print(f"   - Mes: '{month_col}'")
    print(f"   - Canal: '{canal_col}'")
    print(f"   - Sesiones: '{sessions_col}'")
    print(f"   - Rebotes: '{bounces_col}'")
    print(f"   - Duración: '{duration_col}'")
    print(f"   - Vistas: '{views_col}'")

## 3. Inspección inicial

In [ ]:
# Primeras filas
print("📋 Primeras 10 filas del dataset:\n")
df_raw.head(10)

In [ ]:
# Información del dataset
print("ℹ️ Información del dataset:\n")
df_raw.info()

In [ ]:
# Nombres de columnas originales
print("📝 Nombres de columnas originales:\n")
for i, col in enumerate(df_raw.columns, 1):
    print(f"{i}. {col}")

In [ ]:
# Estadísticas descriptivas
print("📊 Estadísticas descriptivas:\n")
df_raw.describe()

## 4. Calidad de datos

In [ ]:
# Verificar valores faltantes
print("🔍 Análisis de valores faltantes:\n")
missing = df_raw.isnull().sum()
missing_pct = (missing / len(df_raw)) * 100

missing_df = pd.DataFrame({
    'Columna': missing.index,
    'Valores Faltantes': missing.values,
    'Porcentaje (%)': missing_pct.values
})

if missing_df['Valores Faltantes'].sum() > 0:
    display(missing_df[missing_df['Valores Faltantes'] > 0])
else:
    print("✅ No hay valores faltantes en el dataset")

In [ ]:
# Verificar duplicados
duplicados = df_raw.duplicated().sum()
print(f"🔍 Registros duplicados: {duplicados}")

if duplicados > 0:
    print("\n⚠️ Mostrando filas duplicadas:")
    display(df_raw[df_raw.duplicated(keep=False)].sort_values(by=df_raw.columns.tolist()))
else:
    print("✅ No hay registros duplicados")

In [ ]:
# Verificar valores únicos en columna de canal
print(f"🎯 Canales únicos encontrados: {df_raw[canal_col].nunique()}\n")
print("Distribución de registros por canal:")
print(df_raw[canal_col].value_counts().sort_index())

In [ ]:
# Verificar rango de meses
print("📅 Rango temporal del dataset:\n")
print(f"Año(s): {sorted(df_raw[year_col].unique())}")
print(f"Meses: {sorted(df_raw[month_col].unique())}")
print(f"\nTotal de meses únicos: {df_raw[[year_col, month_col]].drop_duplicates().shape[0]}")

# Verificar que cada mes tenga datos para todos los canales
registros_por_mes = df_raw.groupby([year_col, month_col])[canal_col].count()
print(f"\nRegistros por mes:")
print(registros_por_mes)

## 5. Análisis por canal

In [ ]:
# Resumen de métricas por canal
print("📊 Resumen de métricas por canal:\n")

resumen = df_raw.groupby(canal_col).agg({
    sessions_col: ['sum', 'mean', 'min', 'max'],
    bounces_col: ['sum', 'mean'],
    duration_col: ['sum', 'mean'],
    views_col: ['sum', 'mean']
}).round(2)

display(resumen)

In [ ]:
# Calcular métricas derivadas por canal para análisis
print("📈 Métricas derivadas promedio por canal:\n")

df_metricas = df_raw.copy()
df_metricas['bounce_rate'] = (df_metricas[bounces_col] / df_metricas[sessions_col]) * 100
df_metricas['views_per_session'] = df_metricas[views_col] / df_metricas[sessions_col]
df_metricas['avg_session_duration'] = df_metricas[duration_col] / df_metricas[sessions_col]

metricas_resumen = df_metricas.groupby(canal_col).agg({
    'bounce_rate': 'mean',
    'views_per_session': 'mean',
    'avg_session_duration': 'mean'
}).round(2)

metricas_resumen.columns = ['Bounce Rate (%)', 'Vistas/Sesión', 'Duración Promedio (seg)']
display(metricas_resumen)

## 6. Visualizaciones exploratorias

In [ ]:
# Crear columna de fecha para visualizaciones
df_raw['fecha'] = pd.to_datetime(
    df_raw[year_col].astype(str) + '-' + df_raw[month_col].astype(str) + '-01'
)
print("✅ Columna de fecha creada")

In [ ]:
# Gráfico 1: Sesiones por canal a lo largo del tiempo
fig, ax = plt.subplots(figsize=(14, 6))

for canal in sorted(df_raw[canal_col].unique()):
    data_canal = df_raw[df_raw[canal_col] == canal].sort_values('fecha')
    ax.plot(data_canal['fecha'], data_canal[sessions_col], 
            marker='o', label=canal, linewidth=2, markersize=6)

ax.set_xlabel('Mes', fontsize=12)
ax.set_ylabel('Sesiones', fontsize=12)
ax.set_title('Evolución de Sesiones por Canal (2025)', fontsize=14, fontweight='bold')
ax.legend(title='Canal', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()

# Guardar gráfico
plt.savefig(RESULTS_FIGURES / 'sessions_by_channel_timeseries.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"💾 Gráfico guardado en: {RESULTS_FIGURES / 'sessions_by_channel_timeseries.png'}")

In [ ]:
# Gráfico 2: Distribución de sesiones por canal (boxplot)
fig, ax = plt.subplots(figsize=(12, 6))

df_raw.boxplot(column=sessions_col, by=canal_col, ax=ax)
ax.set_xlabel('Canal', fontsize=12)
ax.set_ylabel('Sesiones', fontsize=12)
ax.set_title('Distribución de Sesiones por Canal', fontsize=14, fontweight='bold')
plt.suptitle('')  # Remover título automático de pandas
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Guardar gráfico
plt.savefig(RESULTS_FIGURES / 'sessions_distribution_boxplot.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"💾 Gráfico guardado en: {RESULTS_FIGURES / 'sessions_distribution_boxplot.png'}")

In [ ]:
# Gráfico 3: Comparación de métricas totales por canal
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

metricas = [
    (sessions_col, 'Sesiones Totales'),
    (bounces_col, 'Rebotes Totales'),
    (views_col, 'Vistas Totales'),
    (duration_col, 'Duración Total (segundos)')
]

for idx, (metrica, titulo) in enumerate(metricas):
    ax = axes[idx // 2, idx % 2]
    data_canal = df_raw.groupby(canal_col)[metrica].sum().sort_values(ascending=True)
    data_canal.plot(kind='barh', ax=ax, color='steelblue')
    ax.set_title(titulo, fontsize=12, fontweight='bold')
    ax.set_xlabel('Total', fontsize=10)
    ax.set_ylabel('')
    ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()

# Guardar gráfico
plt.savefig(RESULTS_FIGURES / 'metrics_comparison_by_channel.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"💾 Gráfico guardado en: {RESULTS_FIGURES / 'metrics_comparison_by_channel.png'}")

## 7. Conclusiones preliminares

In [ ]:
# Resumen ejecutivo
print("="*70)
print("📊 RESUMEN EJECUTIVO DEL ANÁLISIS EXPLORATORIO")
print("="*70)

print(f"\n1. DIMENSIONES DEL DATASET:")
print(f"   - Total de registros: {df_raw.shape[0]}")
print(f"   - Total de columnas: {df_raw.shape[1]}")
print(f"   - Canales únicos: {df_raw[canal_col].nunique()}")
print(f"   - Meses cubiertos: {df_raw[[year_col, month_col]].drop_duplicates().shape[0]}")

print(f"\n2. CALIDAD DE DATOS:")
print(f"   - Valores faltantes: {df_raw.isnull().sum().sum()}")
print(f"   - Registros duplicados: {df_raw.duplicated().sum()}")

print(f"\n3. CANALES IDENTIFICADOS:")
for canal in sorted(df_raw[canal_col].unique()):
    registros = len(df_raw[df_raw[canal_col] == canal])
    sesiones_totales = df_raw[df_raw[canal_col] == canal][sessions_col].sum()
    print(f"   - {canal}: {registros} registros | {sesiones_totales:,.0f} sesiones totales")

print(f"\n4. MÉTRICAS PRINCIPALES (TOTALES):")
print(f"   - Sesiones: {df_raw[sessions_col].sum():,.0f}")
print(f"   - Rebotes: {df_raw[bounces_col].sum():,.0f}")
print(f"   - Vistas: {df_raw[views_col].sum():,.0f}")
print(f"   - Duración total: {df_raw[duration_col].sum():,.2f} segundos")

print(f"\n5. OBSERVACIONES:")
canales_bajo_volumen = df_raw.groupby(canal_col)[sessions_col].mean()
canales_bajo_volumen = canales_bajo_volumen[canales_bajo_volumen < 1000].index.tolist()
if canales_bajo_volumen:
    print(f"   ⚠️ Canales con bajo volumen (<1000 sesiones promedio):")
    for canal in canales_bajo_volumen:
        print(f"      - {canal}")
    print(f"   💡 Considerar si estos canales deben incluirse en el forecasting")
else:
    print(f"   ✅ Todos los canales tienen volumen significativo")

print(f"\n6. PRÓXIMOS PASOS:")
print(f"   ✓ Limpieza y transformación de datos (Notebook 02)")
print(f"   ✓ Convertir nombres de columnas a snake_case")
print(f"   ✓ Calcular métricas derivadas")
print(f"   ✓ Decidir qué canales incluir en el forecasting")

print("\n" + "="*70)

---

## Notas importantes:

- Este notebook asume que el CSV está en `data/raw/ga4_promtur_organic_2025.csv`
- Todos los nombres de columnas están centralizados en la celda de configuración (sección 2.1)
- Las visualizaciones se guardan automáticamente en `results/figures/exploratory/`
- Si tienes datos de 2024, repite este análisis para ese año

## ✅ Checklist antes de continuar al Notebook 02:

- [ ] CSV cargado correctamente
- [ ] Sin valores faltantes críticos
- [ ] Canales identificados y validados
- [ ] Visualizaciones generadas y revisadas
- [ ] Decisión tomada sobre qué canales incluir en el análisis